In [8]:
import json
import re
import logging
from typing import Dict, Any

# Set up a dummy logger to avoid errors
logger = logging.getLogger(__name__)
logger.addHandler(logging.NullHandler())

# Define the function (assuming it's part of a class, but we'll test it standalone)
def _parse_response(response: str) -> Dict[str, Any]:
    """Parse the LLM response into a structured format.
    
    Args:
        response: LLM response string
    
    Returns:
        Dictionary with parsed response
    """
    try:
        # Try to parse as JSON
        data = json.loads(response)
        
        # Ensure required fields are present
        if "thought" not in data:
            data["thought"] = "No thought provided"
        if "action" not in data:
            data["action"] = "no_action"
        if "parameters" not in data:
            data["parameters"] = {}
            
        return data
    except json.JSONDecodeError:
        logger.warning(f"Initial JSON parsing failed for response: {response[:100]}...") # Log the failure
        # If not valid JSON, try to extract fields using regex
        thought_match = re.search(r'thought["\s:]+([^"]+)', response)
        action_match = re.search(r'action["\s:]+([^",\n]+)', response)
        # Modified regex slightly to be more robust for parameters block
        # Look for parameters block starting with { and ending with } potentially across lines
        params_match = re.search(r'parameters["\s:]+({.*?})', response, re.DOTALL) 
        
        thought = thought_match.group(1).strip() if thought_match else "Failed to parse thought"
        action = action_match.group(1).strip() if action_match else "no_action"
        
        # Try to parse parameters
        params = {}
        if params_match:
            try:
                params_str = params_match.group(1)
                # Replace single quotes with double quotes for valid JSON (optional here, but good practice)
                params_str = params_str.replace("'", "\"")
                # Attempt to clean potential markdown/extra chars if needed - basic attempt
                # params_str = re.sub(r'^```json\s*|\s*```$', '', params_str).strip() 
                params = json.loads(params_str)
            except json.JSONDecodeError as e_params:
                logger.warning(f"Failed to parse parameters JSON from regex match: {params_str}. Error: {e_params}")
                params = {"error": "Failed to parse parameters string", "raw_params": params_str} # Store raw if fails
        
        return {
            "thought": thought,
            "action": action,
            "parameters": params
        }

# The actual input string including the markdown fences
input_string = """"```json\n{\n  \"thought\": \"The current screen does not show the Grok Beta app icon. I need to go back to the home screen to find the app icon. I will use the `press_key` tool with the keycode for the Home button (keycode 3). After returning to the home screen, I will use `get_clickables` to find the app icon.\",\n  \"action\": \"press_key\",\n  \"parameters\": {\n    \"keycode\": 3\n  }\n}\n```"""

# Call the function and print the result
reasoning_result = _parse_response(input_string)


In [7]:
thought = reasoning_result.get("thought", "")
action = reasoning_result.get("action", "")
parameters = reasoning_result.get("parameters", {})



# Execute the action if it's a valid tool
result = "No action taken"
print(action)
print(f"Parameters: {parameters}")


tap
Parameters: {'index': 27}


In [9]:
reasoning_result

{'thought': 'The current screen does not show the Grok Beta app icon. I need to go back to the home screen to find the app icon. I will use the `press_key` tool with the keycode for the Home button (keycode 3). After returning to the home screen, I will use `get_clickables` to find the app icon.',
 'action': 'press_key',
 'parameters': {'keycode': 3}}

In [10]:
import builtins
import sys
from typing import Any, Dict, Tuple

# Assuming SimpleCodeExecutor is in executor.py
from typing import Any, Dict, Tuple
import io
import contextlib
import ast
import traceback


class SimpleCodeExecutor:
    """
    A simple code executor that runs Python code with state persistence.

    This executor maintains a global and local state between executions,
    allowing for variables to persist across multiple code runs.

    NOTE: not safe for production use! Use with caution.
    """

    def __init__(self, locals: Dict[str, Any], globals: Dict[str, Any]):
        """
        Initialize the code executor.

        Args:
            locals: Local variables to use in the execution context
            globals: Global variables to use in the execution context
        """
        # State that persists between executions
        self.globals = globals
        self.locals = locals

    def execute(self, code: str) -> Tuple[bool, str, Any]:
        """
        Execute Python code and capture output and return values.

        Args:
            code: Python code to execute

        Returns:
            Dict with keys `success`, `output`, and `return_value`
        """
        # Capture stdout and stderr
        stdout = io.StringIO()
        stderr = io.StringIO()

        output = ""
        return_value = None
        try:
            # Execute with captured output
            with contextlib.redirect_stdout(
                stdout
            ), contextlib.redirect_stderr(stderr):
                # Try to detect if there's a return value (last expression)
                try:
                    tree = ast.parse(code)
                    last_node = tree.body[-1] if tree.body else None

                    # If the last statement is an expression, capture its value
                    if isinstance(last_node, ast.Expr):
                        # Split code to add a return value assignment
                        last_line = code.rstrip().split("\n")[-1]
                        exec_code = (
                            code[: -len(last_line)]
                            + "\n__result__ = "
                            + last_line
                        )

                        # Execute modified code
                        exec(exec_code, self.globals, self.locals)
                        return_value = self.locals.get("__result__")
                    else:
                        # Normal execution
                        exec(code, self.globals, self.locals)
                except:
                    # If parsing fails, just execute the code as is
                    exec(code, self.globals, self.locals)

            # Get output
            output = stdout.getvalue()
            if stderr.getvalue():
                output += "\n" + stderr.getvalue()

        except Exception as e:
            # Capture exception information
            output = f"Error: {type(e).__name__}: {str(e)}\n"
            output += traceback.format_exc()

        if return_value is not None:
            output += "\n\n" + str(return_value)

        return output

# --- Helper function to print executor output clearly ---
def print_execution_result(description: str, result: str):
    print(f"--- {description} ---")
    print(result)
    print("-" * (len(description) + 6))


# --- Setup the executor ---

# Method 1: Initialize with caller's globals (most common for interactive sessions)
# This makes __builtins__ automatically available
# initial_globals = globals().copy()
# initial_locals = {}
# executor = SimpleCodeExecutor(locals=initial_locals, globals=initial_globals)

# Method 2: Explicitly provide minimal globals including builtins
# This gives more control over what's in the global scope
initial_globals_explicit = {'__builtins__': builtins}
initial_locals_explicit = {}
executor = SimpleCodeExecutor(locals=initial_locals_explicit, globals=initial_globals_explicit)

print("Initial Executor State (globals & locals):")
print("Globals:", list(executor.globals.keys())) # Show keys, not full dict
print("Locals:", list(executor.locals.keys()))
print("\n" + "="*30 + "\n")

# --- Test 1: Import and Use 'math' ---
print("Attempting to import and use 'math'...")
math_code = """
import math
print(f"Pi is approximately: {math.pi}")
result = math.sqrt(16)
print(f"Square root of 16 is: {result}")
math_result = result * 10
"""
math_output = executor.execute(math_code)
print_execution_result("Math Import and Usage", math_output)

print("\nExecutor State After Math Import:")
print("Globals:", list(executor.globals.keys()))
print("Locals:", list(executor.locals.keys())) # Expect 'math' and 'math_result' here
print("\n" + "="*30 + "\n")

# Test persistence: Use the imported 'math' from the previous step
print("Attempting to use 'math' in a subsequent execution...")
math_persistence_code = """
# math was imported in the previous step and should be in locals
print(f"Using math.ceil(4.5): {math.ceil(4.5)}")
# math_result was defined in the previous step
print(f"Previous math result was: {math_result}")
"""
math_persistence_output = executor.execute(math_persistence_code)
print_execution_result("Math Persistence Test", math_persistence_output)

print("\n" + "="*30 + "\n")

# --- Test 2: Import and Use 'matplotlib' ---
# This will only work if matplotlib is installed!
print("Attempting to import 'matplotlib.pyplot'...")
matplotlib_code_import = """
import matplotlib.pyplot as plt
print("Successfully imported matplotlib.pyplot as plt")
"""
matplotlib_output_import = executor.execute(matplotlib_code_import)
print_execution_result("Matplotlib Import", matplotlib_output_import)

print("\nExecutor State After Matplotlib Import:")
print("Globals:", list(executor.globals.keys()))
print("Locals:", list(executor.locals.keys())) # Expect 'plt' here if successful
print("\n" + "="*30 + "\n")

# Test persistence: Use the imported 'matplotlib.pyplot' from the previous step
# This requires a non-interactive backend for pyplot if not running in a GUI environment
# We'll just do a simple check for now
print("Attempting to use 'matplotlib.pyplot' in a subsequent execution...")
matplotlib_persistence_code = """
# plt was imported in the previous step and should be in locals
print(f"Type of plt: {type(plt)}")
# Creating a simple plot (won't display unless backend is configured)
# plt.figure()
# plt.plot([1, 2, 3], [5, 7, 4])
# plt.title("Test Plot")
# print("Attempted to create a matplotlib plot")
"""
matplotlib_persistence_output = executor.execute(matplotlib_persistence_code)
print_execution_result("Matplotlib Persistence Test", matplotlib_persistence_output)

print("\n" + "="*30 + "\n")

# --- Test 3: Attempt to import a non-existent library ---
print("Attempting to import a non-existent library...")
nonexistent_code = """
import nonexistent_library_xyz_123
print("This line should not be reached")
"""
nonexistent_output = executor.execute(nonexistent_code)
print_execution_result("Non-existent Library Import Test", nonexistent_output)

Initial Executor State (globals & locals):
Globals: ['__builtins__']
Locals: []


Attempting to import and use 'math'...
--- Math Import and Usage ---
Pi is approximately: 3.141592653589793
Square root of 16 is: 4.0

---------------------------

Executor State After Math Import:
Globals: ['__builtins__']
Locals: ['math', 'result', 'math_result']


Attempting to use 'math' in a subsequent execution...
--- Math Persistence Test ---
Using math.ceil(4.5): 5
Using math.ceil(4.5): 5
Previous math result was: 40.0

---------------------------


Attempting to import 'matplotlib.pyplot'...
--- Matplotlib Import ---
Successfully imported matplotlib.pyplot as plt

-----------------------

Executor State After Matplotlib Import:
Globals: ['__builtins__']
Locals: ['math', 'result', 'math_result', 'plt']


Attempting to use 'matplotlib.pyplot' in a subsequent execution...
--- Matplotlib Persistence Test ---
Type of plt: <class 'module'>

---------------------------------


Attempting to import a non

In [11]:
nonexistent_code = """
import numpy as np
print("This line should not be reached")
"""
nonexistent_output = executor.execute(nonexistent_code)
print_execution_result("Non-existent Library Import Test", nonexistent_output)

--- Non-existent Library Import Test ---
This line should not be reached

--------------------------------------


In [13]:
executor.locals.keys()

dict_keys(['math', 'result', 'math_result', 'plt', 'np'])

In [3]:
from tools import *

# set environment variables for `DROIDRUN_DEVICE_SERIAL` to R5CY10VM7YT
import asyncio
import os

async def main():
    packages = await list_packages(serial="192.168.0.17:5555")
    print(packages.keys())

await main()

ModuleNotFoundError: No module named 'tools'

In [ ]:
device_manager = DeviceManager()
device = await device_manager.connect("192.168.0.17")

In [19]:
device